In [2]:
import os

os.chdir('../..')
print(os.getcwd())

/workspace/LLaSA


In [3]:
# reload module 
import importlib
import torch
from transformers import BitsAndBytesConfig, LlavaNextVideoForConditionalGeneration
from model.llasa_arch import LLaSA
from model.llasa_processor import LlavaNextVideoProcessor


importlib.reload(importlib.import_module("model.llasa_arch"))
importlib.reload(importlib.import_module("model.llasa_processor"))

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16
)

#del processor
#del model

processor = LlavaNextVideoProcessor.from_pretrained("/workspace/LLaSA/checkpoints/LLaVA-NeXT-Video-7B-hf", cache_dir='/workspace/LLaSA/checkpoints')
model = LLaSA.from_pretrained(
    "/workspace/LLaSA/checkpoints/LLaVA-NeXT-Video-7B-hf",
    quantization_config=quantization_config,
    device_map='auto',
    cache_dir = '/workspace/LLaSA/checkpoints'
)


/opt/conda/envs/llasa/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Unrecognized keys in `rope_scaling` for 'rope_type'='linear': {'type'}
Loading checkpoint shards: 100%|██████████| 3/3 [00:08<00:00,  2.98s/it]
Some weights of LLaSA were not initialized from the model checkpoint at /workspace/LLaSA/checkpoints/LLaVA-NeXT-Video-7B-hf and are newly initialized: ['seg_projector.linear.bias', 'seg_projector.linear.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [5]:
import av
import numpy as np

def read_video_pyav(container, indices):
    '''
    Decode the video with PyAV decoder.

    Args:
        container (av.container.input.InputContainer): PyAV container.
        indices (List[int]): List of frame indices to decode.

    Returns:
        np.ndarray: np array of decoded frames of shape (num_frames, height, width, 3).
    '''
    frames = []
    container.seek(0)
    start_index = indices[0]
    end_index = indices[-1]
    for i, frame in enumerate(container.decode(video=0)):
        if i > end_index:
            break
        if i >= start_index and i in indices:
            frames.append(frame)
    return np.stack([x.to_ndarray(format="rgb24") for x in frames])

In [25]:
from huggingface_hub import hf_hub_download

# Download video from the hub
#video_path_1 = hf_hub_download(repo_id="raushan-testing-hf/videos-test", filename="sample_demo_1.mp4", repo_type="dataset")
#ideo_path_2 = hf_hub_download(repo_id="raushan-testing-hf/videos-test", filename="karate.mp4", repo_type="dataset")

video_path_1 = '/workspace/LLaSA/dataset/A2D/clips320H/__KkKB4wzrY.mp4'

container = av.open(video_path_1)

# sample uniformly 8 frames from the video (we can sample more for longer videos)
total_frames = container.streams.video[0].frames
indices = np.arange(0, total_frames, total_frames / 8).astype(int)
clip_baby = read_video_pyav(container, indices)

"""
container = av.open(video_path_2)

# sample uniformly 8 frames from the video (we can sample more for longer videos)
total_frames = container.streams.video[0].frames
indices = np.arange(0, total_frames, total_frames / 8).astype(int)
clip_karate = read_video_pyav(container, indices)"""

'\ncontainer = av.open(video_path_2)\n\n# sample uniformly 8 frames from the video (we can sample more for longer videos)\ntotal_frames = container.streams.video[0].frames\nindices = np.arange(0, total_frames, total_frames / 8).astype(int)\nclip_karate = read_video_pyav(container, indices)'

In [26]:
clip_baby.dtype

dtype('uint8')

In [ ]:
from matplotlib import pyplot as plt
from matplotlib import animation
from IPython.display import HTML

# np array with shape (frames, height, width, channels)
video = clip_baby

fig = plt.figure()
im = plt.imshow(video[0,:,:,:])

plt.close() # this is required to not display the generated image

def init():
    im.set_data(video[0,:,:,:])

def animate(i):
    im.set_data(video[i,:,:,:])
    return im

#anim = animation.FuncAnimation(fig, animate, init_func=init, frames=video.shape[0],
#                               interval=100)
#HTML(anim.to_html5_video())

In [ ]:
# Lets also load 2 images for generation from image data

from PIL import Image
import requests

image_stop = Image.open(requests.get("https://www.ilankelman.org/stopsigns/australia.jpg", stream=True).raw)
image_snowman = Image.open(requests.get("https://huggingface.co/microsoft/kosmos-2-patch14-224/resolve/main/snowman.jpg", stream=True).raw)

In [7]:
# Each "content" is a list of dicts and you can add image/video/text modalities
conversation = [
      {
          "role": "user",
          "content": [
              {"type": "text", "text": "Why is this video funny?"},
              {"type": "video"},
              {"type": "seg"},
              ],
      },
]

conversation_2 = [
      {
          "role": "user",
          "content": [
              {"type": "text", "text": "What do you see in this video?"},
              {"type": "video"},
              ],
      },
]

prompt = processor.apply_chat_template(conversation, add_generation_prompt=True)
#prompt_2 = processor.apply_chat_template(conversation_2, add_generation_prompt=True)


In [ ]:
processor.chat_template

In [ ]:
inputs = processor([prompt, prompt_2], videos=[clip_baby, clip_karate], padding=True, return_tensors="pt").to(model.device)

In [8]:
import torch
seg = torch.ones(1, 4096, 256)
seg.shape
seg = seg.transpose(1,2)
print(seg.shape)
seg = seg.to(torch.float16)

torch.Size([1, 256, 4096])


In [ ]:
print(seg)

In [ ]:
clip_baby.shape

In [17]:
prompt

'USER: <video>\nWhy is this video funny? <seg>\nASSISTANT:'

In [27]:
#inputs = processor([prompt], videos=[clip_baby], padding=True, return_tensors="pt").to(model.device)
inputs = processor([prompt], videos=[clip_baby], seg=seg, labels='sda',  padding=True, return_tensors="pt").to(model.device)

In [23]:
model.device

device(type='cuda', index=0)

In [21]:
inputs

{'input_ids': tensor([[   1,  269, 1388]], device='cuda:0'), 'attention_mask': tensor([[1, 1, 1]], device='cuda:0'), 'pixel_values_videos': tensor([[[[[ 9.9604e-01,  9.9604e-01,  9.9604e-01,  ...,  3.8290e-01,
             3.8290e-01,  3.8290e-01],
           [ 1.0106e+00,  1.0106e+00,  1.0106e+00,  ...,  3.8290e-01,
             3.8290e-01,  3.8290e-01],
           [ 1.0106e+00,  1.0106e+00,  1.0106e+00,  ...,  3.8290e-01,
             3.8290e-01,  3.8290e-01],
           ...,
           [-1.6901e+00, -1.6901e+00, -1.6901e+00,  ..., -1.7923e+00,
            -1.7923e+00, -1.7923e+00],
           [-1.6901e+00, -1.6901e+00, -1.6901e+00,  ..., -1.7923e+00,
            -1.7923e+00, -1.7923e+00],
           [-1.6901e+00, -1.6901e+00, -1.6901e+00,  ..., -1.7923e+00,
            -1.7923e+00, -1.7923e+00]],

          [[ 1.1444e+00,  1.1444e+00,  1.1444e+00,  ...,  5.5910e-01,
             5.5910e-01,  5.5910e-01],
           [ 1.1594e+00,  1.1594e+00,  1.1594e+00,  ...,  5.5910e-01,
         

In [30]:
inputs['pixel_values_videos'].dtype

torch.float32

In [31]:
generate_kwargs = {"max_new_tokens": 100, "do_sample": True, "top_p": 0.9}

output = model.generate(**inputs, **generate_kwargs)
generated_text = processor.batch_decode(output, skip_special_tokens=False)

ValueError: Number of image tokens in input_ids (0) different from num_images (1).

In [21]:
!pip install pynvml

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [22]:
from pynvml import *


def print_gpu_utilization():
    nvmlInit()
    handle = nvmlDeviceGetHandleByIndex(0)
    info = nvmlDeviceGetMemoryInfo(handle)
    print(f"GPU memory occupied: {info.used//1024**2} MB.")


def print_summary(result):
    print(f"Time: {result.metrics['train_runtime']:.2f}")
    print(f"Samples/second: {result.metrics['train_samples_per_second']:.2f}")
    print_gpu_utilization()

In [23]:
from transformers import TrainingArguments, Trainer, logging
default_args = {
    "output_dir": "tmp",
    "eval_strategy": "steps",
    "num_train_epochs": 1,
    "log_level": "error",
    "report_to": "none",
}

logging.set_verbosity_error()


training_args = TrainingArguments(per_device_train_batch_size=4, **default_args)
trainer = Trainer(model=model, args=training_args, train_dataset=ds)
result = trainer.train()
print_summary(result)

NameError: name 'ds' is not defined

In [18]:
print(generated_text)

['<s> USER: <video>\nWhy is this video funny? <seg>\nASSISTANT:\nod? groups in the equipments available. The keys availables?s are nots in the meanings, The birds?</s>']


In [ ]:
model.config['seg_tok
print(model.config)

In [ ]:
print(model.config)

In [ ]:
!pip install h5py

In [ ]:

from scipy.io import loadmat

# Load the .mat file
data = loadmat()

In [ ]:
import h5py
import torch
import numpy as np

# Open the .mat file
with h5py.File('/workspace/LLaSA/dataset/A2D/a2d_annotation_with_instances/ouFmCnCh1wk/00030.h5', 'r') as f:
    # List all variables stored in the .mat file
    print("Keys in the file:", list(f.keys()))


    def print_structure(name, obj):
        """Print the structure of the HDF5 file"""
        if isinstance(obj, h5py.Dataset):
            print(f"Dataset: {name}, Shape: {obj.shape}, Data type: {obj.dtype}")
        elif isinstance(obj, h5py.Group):
            print(f"Group: {name}")

    # Visit each object in the file
    f.visititems(print_structure)

    coord = f['reBBox'][:]
    print(f['instance'][:])



In [ ]:
if input_ids.shape[1] != 1:
    iterator = (
        (image_features, feature_lens, self.config.image_token_index),
        (video_features, video_feature_lens, self.config.video_token_index),
    )
    for features, lens, special_token in iterator:
        if features is not None:
            print('before:',
                inputs_embeds.shape,
                attention_mask.shape,
                position_ids,
                labels,
                input_ids)
            (
                inputs_embeds,
                attention_mask,
                position_ids,
                labels,
                input_ids,
            ) = self._merge_input_ids_with_image_features(
                features,
                lens,
                inputs_embeds,
                input_ids,
                attention_mask,
                position_ids,
                labels=labels,
                image_token_index=special_token,
            )
            print('after:',
                inputs_embeds.shape,
                attention_mask.shape,
                position_ids,
                labels,
                input_ids)

In [ ]:
!pip install --no-cache-dir opencv-contrib-python 
!pip install  --no-cache-dir opencv-python
!pip install --no-cache-dir opencv-python-headless


In [ ]:
!pip install --upgrade opencv-python

In [ ]:
from PIL import Image
import matplotlib.pyplot as plt
import matplotlib.patches as patches

def draw_bounding_box(image_path, bbox):
    """
    Retrieve an image from a local path and draw a bounding box on it using matplotlib.

    :param image_path: Path to the local image file
    :param bbox: Bounding box coordinates in the format (x_min, y_min, x_max, y_max)
    """
    # Load the image using PIL
    image = Image.open(image_path)

    # Create a figure and axis
    fig, ax = plt.subplots(1)

    # Display the image
    ax.imshow(image)

    # Extract bounding box coordinates
    x_min, y_min, x_max, y_max = bbox

    # Create a Rectangle patch
    rect = patches.Rectangle((x_min, y_min), x_max - x_min, y_max - y_min,
                             linewidth=2, edgecolor='g', facecolor='none')

    # Add the rectangle to the plot
    ax.add_patch(rect)

    # Show the plot
    plt.show()

# Example usage
image_path = '/workspace/LLaSA/dataset/A2D/clips320jpeg/ouFmCnCh1wk/00030.jpeg'  # Replace with the path to your image file
num = 2
bbox = tuple(sub_coord[num] for sub_coord in coord)  # Example bounding box coordinates (x_min, y_min, x_max, y_max)

draw_bounding_box(image_path, bbox)


In [ ]:
import json
import torch
from torch.utils.data import Dataset, DataLoader

class VideoCaptionDataset(Dataset):
    def __init__(self, data, pt_file_dir, start_idx, end_idx):
        """
        Args:
            data (dict): A dictionary containing video metadata.
            pt_file_dir (str): Directory path where .pt files are stored.
            start_idx (int): Starting index of the keys to include in the dataset.
            end_idx (int): Ending index (exclusive) of the keys to include in the dataset.
        """
        self.pt_file_dir = pt_file_dir
        self.keys = list(data.keys())[start_idx:end_idx]  # Get the subset of keys
        self.data = {key: data[key] for key in self.keys}  # Subset of data

    def __len__(self):
        return len(self.keys)

    def __getitem__(self, idx):
        key = self.keys[idx]
        item = self.data[key]
        
        # Extract data
        caption = item['caption']
        bbox = torch.tensor(item['bbox'])  # Convert bbox to tensor
        video_id = item['video']
        pt_file_path = f"{self.pt_file_dir}/{video_id}.pt"  # Assuming .pt files are named after video IDs

        # Load the pt file (this will load a tensor)
        features = torch.load(pt_file_path)

        return caption, bbox, features, video_id

# Load JSON data
json_file_path = 'path/to/your/data.json'  # Path to your JSON file
pt_file_dir = 'path/to/pt/files'           # Directory containing .pt files

with open(json_file_path, 'r') as f:
    data = json.load(f)

# Create training and testing datasets
train_dataset = VideoCaptionDataset(data, pt_file_dir, start_idx=0, end_idx=2000)
test_dataset = VideoCaptionDataset(data, pt_file_dir, start_idx=2000, end_idx=2500)

# Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

# Iterate over the train DataLoader
for batch in train_loader:
    captions, bboxes, features, video_ids = batch
    print("Training batch - Captions:", captions)
    print("Training batch - Bounding boxes:", bboxes)
    print("Training batch - Features shape:", features.shape)
    print("Training batch - Video IDs:", video_ids)
    break  # Just show the first batch for demonstration

# Iterate over the test DataLoader
for batch in test_loader:
    captions, bboxes, features, video_ids = batch
    print("Testing batch - Captions:", captions)
    print("Testing batch - Bounding boxes:", bboxes)
    print("Testing batch - Features shape:", features.shape)
    print("Testing batch - Video IDs:", video_ids)
    break  # Just show the first batch for demonstratio
